Смоделировать игру против лотерейного автомата типа “777”. Игрок платит 1 руб.,
после чего выпадает случайное целое число, равномерно распределенное от 0 до 999.
При некоторых значениях числа игрок получает выигрыш (см. справа)

Выгодна ли игра игроку?

Сколько в среднем приобретает
или теряет игрок за одну игру?

- 777: 200 руб.
- 999: 100 руб.
- 555: 50 руб.
- 333: 15 руб.
- 111: 10 руб.
- *77: 5 руб.
- **7: 3 руб.
- *00: 2 руб.
- **0: 1 руб.

* – любая цифра


**Дополнительное задание повышенной сложности.**

Теоретически рассчитать средний выигрыш (проигрыш) и сравнить с результатами моделирования

In [1]:
import numpy as np
import re


In [2]:
winning_nums =  {
    777: 200, 999: 100, 555: 50, 333: 15, 111: 10
}

winning_lookups = [
    (r'([012345689]{0,2})(?<!7)77$', 5), # *77
    (r'([0123456789]{1,2})(?<!7)7$', 3), # **7
    (r'([012345689]{0,2})(?<!0)00$', 2),  # *00
    (r'([012345689]{0,2})(?<!0)0$', 1)  # **0
]

def gamble():
    global gamer_payed
    global gamer_won
    gamer_payed += 1

    result = np.random.randint(0, 999)
    # result = 777
    # print(f"Your numbers are: {result}")

    if result in winning_nums:
        gamer_won += winning_nums[result]
        # print(f"Congrats! You won: {winning_nums[result]}. Total: {gamer_won}")
        return gamer_won, gamer_payed
    else:
        for pattern, value in winning_lookups:
            if re.match(pattern, str(result)):
                gamer_won += value
                # print(f"Congrats! You won: {value}. Total: {gamer_won}")
                return gamer_won, gamer_payed

        # print(f'Sorry, you lost. '
        #       f'You spent: {gamer_payed}, you won: {gamer_won}')
    return gamer_won, gamer_payed

In [3]:
iterations = 1000000
gamer_payed = 0
gamer_won = 0

for i in range(iterations):
    gamble()

print(
    f"Игр: {iterations} \n"
    f"Потрачено денег: {gamer_payed} \n"
    f"Выиграно денег: {gamer_won}\n"
    f"Игрок приобретает в среднем за 1 игру: {(gamer_won-gamer_payed)/ iterations} денег"
)

# Игра не выгодна, при увеличении количества интераций средний проигрыш колеблется в диапозоне 0,3 - 0,32 рублей
# При итерациях меньше 1000 игра иногда может быть выгодной

Игр: 1000000 
Потрачено денег: 1000000 
Выиграно денег: 678513
Игрок приобретает в среднем за 1 игру: -0.321487 денег


## Теоретический расчет среднего выигрыша

События:

- P(выигрыша) = количество всех выигрышных комбинаций/ все возможные комбинации
- P(проигрыша) = 1 - P(выигрыша)

- P(любая цифра) = 1 или 100%

- P(конкретная цифра) = 0,1 - 10%, один шанс из 10
- P(все кроме конкретной цифры) = 1- P(конкретная цифра) = 1 - 0,1

- P(&lowast;&lowast;7) = 1&lowast;1&lowast;0,1
- P(&lowast;77) =  = 1&lowast;0,1&lowast;0,1
- P(777)= P(конкретная цифра) &lowast; P(конкретная цифра) &lowast; P(конкретная цифра) = 0,1&lowast;0,1&lowast;0,1

In [4]:
prob_777 = 0.1*0.1*0.1 #вероятность получения 200р
prob_999 = 0.1*0.1*0.1 #вероятность получения 100р
prob_555 = 0.1*0.1*0.1 #вероятность получения 50р
prob_333 = 0.1*0.1*0.1 #вероятность получения 15р
prob_111 = 0.1*0.1*0.1 #вероятность получения 10р
prob_x77 = (1*0.1*0.1) - prob_777 #вероятность получения 5р
prob_xx7 = (1*1*0.1) - prob_777 - prob_x77 #вероятность получения 3р
prob_x00 = 1*0.1*0.1 #вероятность получения 2р
prob_xx0 = (1*1*0.1) - prob_x00 #вероятность получения 1р

prob_winning = (prob_777+ prob_999+ prob_555 + prob_333 +
                prob_111 + prob_x77 + prob_xx7 + prob_x00 + prob_xx0) # вероятность получения выигрыша

average_winning = (
    (prob_777*(200-1))+  # из суммы выигрыша вычитаем затраты на игру
    (prob_999*(100-1))+
    (prob_555*(50-1))+
    (prob_333*(15-1))+
    (prob_111*(10-1))+
    (prob_x77*(5-1))+
    (prob_xx7*(3-1))+
    (prob_x00*(2-1))+
    (prob_xx0*(1-1)) )

average_losing = (1-prob_winning)*(0-1)


print(f"Теоретический чистый средний выигрыш: {average_winning} \n"
      f"Теоретический чистый средний проигрыш: {average_losing} \n"
      f"Выгода: {round(average_winning+average_losing, 3)}")

Теоретический чистый средний выигрыш: 0.5960000000000001 
Теоретический чистый средний проигрыш: -0.796 
Выгода: -0.2


Теоретический расчет среднего выигрыша имеет близкое значение с результатами моделирования.
Несмотря на то, что разница значительная, вывод одинаков: игра не выгодна.